<a href="https://colab.research.google.com/github/VrijKun/CS6910_Assignment_1/blob/main/Assignment_1_DL_ED23D015_q1-q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login() #3c21150eb43b007ee446a1ff6e87f640ec7528c4

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from keras.datasets import fashion_mnist
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load the Fashion-MNIST dataset
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()


image_classes = len(np.unique(Y_train)) #to find out number of unique images
image_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] #taken from keras documentation

def show_images(image_classe,img_labels,X,Y):
   set_Labels=[]
   set_Images = []

   for i in range(len(X)):
     if len(set_Images)== image_classes:
       break
     if image_labels[Y[i]] not in set_Labels:

      set_Labels.append(image_labels[Y[i]])
      set_Images.append(X[i])

   run=wandb.init(project='DL_Assignment_1',entity='ed23d015',reinit=True)
   run.log({"Sample from each class":list(wandb.Image(img,caption= caption) for img,caption in zip(set_Images,set_Labels))})
   run.finish()

#Getting prints
show_images(image_classes,image_labels,X_test,Y_test)

4422102/4422102 [==============================] - 0s 0us/step


wandb: Currently logged in as: ed23d015. Use `wandb login --relogin` to force relogin


In [ ]:
import numpy as np
from keras.datasets import fashion_mnist
from keras.utils import to_categorical


def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def forward_propagation(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    for l in range(1, L):
        A_prev = A
        Z = np.dot(parameters['W' + str(l)], A_prev) + parameters['b' + str(l)]
        A = relu(Z)
        cache = (A_prev, parameters['W' + str(l)], parameters['b' + str(l)], Z)
        caches.append(cache)
    ZL = np.dot(parameters['W' + str(L)], A) + parameters['b' + str(L)]
    AL = softmax(ZL)
    cache = (A, parameters['W' + str(L)], parameters['b' + str(L)], ZL)
    caches.append(cache)
    return AL, caches

def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -np.sum(np.multiply(Y, np.log(AL))) / m
    return cost

def backward_propagation(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    dZL = AL - Y
    A_prev, W, b, Z = caches[L-1]
    grads['dW' + str(L)] = np.dot(dZL, A_prev.T) / m
    grads['db' + str(L)] = np.sum(dZL, axis=1, keepdims=True) / m
    dAL = np.dot(W.T, dZL)
    for l in reversed(range(L-1)):
        A_prev, W, b, Z = caches[l]
        dZ = np.array(dAL, copy=True)
        dZ[Z <= 0] = 0
        grads['dW' + str(l + 1)] = np.dot(dZ, A_prev.T) / m
        grads['db' + str(l + 1)] = np.sum(dZ, axis=1, keepdims=True) / m
        dAL = np.dot(W.T, dZ)
    return grads

def update_parameters(parameters, grads, learning_rate, method="SGD", beta1=0.9, beta2=0.999, epsilon=1e-8):
    L = len(parameters) // 2
    if method == "SGD":
        for l in range(L):
            parameters['W' + str(l+1)] -= learning_rate * grads['dW' + str(l+1)]
            parameters['b' + str(l+1)] -= learning_rate * grads['db' + str(l+1)]
    elif method == "Momentum":
        for l in range(L):
            v_dW = beta1 * grads['dW' + str(l+1)]
            v_db = beta1 * grads['db' + str(l+1)]
            parameters['W' + str(l+1)] -= learning_rate * v_dW
            parameters['b' + str(l+1)] -= learning_rate * v_db
    elif method == "Nesterov":
        for l in range(L):
            v_dW = beta1 * grads['dW' + str(l+1)]
            v_db = beta1 * grads['db' + str(l+1)]
            parameters['W' + str(l+1)] -= learning_rate * (v_dW + beta1 * v_dW)
            parameters['b' + str(l+1)] -= learning_rate * (v_db + beta1 * v_db)
    elif method == "RMSprop":
        for l in range(L):
            s_dW = beta2 * grads['dW' + str(l+1)]**2 + (1 - beta2) * (grads['dW' + str(l+1)]**2)
            s_db = beta2 * grads['db' + str(l+1)]**2 + (1 - beta2) * (grads['db' + str(l+1)]**2)
            parameters['W' + str(l+1)] -= learning_rate * grads['dW' + str(l+1)] / (np.sqrt(s_dW) + epsilon)
            parameters['b' + str(l+1)] -= learning_rate * grads['db' + str(l+1)] / (np.sqrt(s_db) + epsilon)
    elif method == "Adam":
        for l in range(L):
            v_dW = beta1 * grads['dW' + str(l+1)] + (1 - beta1) * grads['dW' + str(l+1)]
            v_db = beta1 * grads['db' + str(l+1)] + (1 - beta1) * grads['db' + str(l+1)]
            s_dW = beta2 * grads['dW' + str(l+1)]**2 + (1 - beta2) * (grads['dW' + str(l+1)]**2)
            s_db = beta2 * grads['db' + str(l+1)]**2 + (1 - beta2) * (grads['db' + str(l+1)]**2)
            parameters['W' + str(l+1)] -= learning_rate * v_dW / (np.sqrt(s_dW) + epsilon)
            parameters['b' + str(l+1)] -= learning_rate * v_db / (np.sqrt(s_db) + epsilon)
    elif method == "Nadam":
        for l in range(L):
            v_dW = beta1 * grads['dW' + str(l+1)] + (1 - beta1) * grads['dW' + str(l+1)]
            v_db = beta1 * grads['db' + str(l+1)] + (1 - beta1) * grads['db' + str(l+1)]
            s_dW = beta2 * grads['dW' + str(l+1)]**2 + (1 - beta2) * (grads['dW' + str(l+1)]**2)
            s_db = beta2 * grads['db' + str(l+1)]**2 + (1 - beta2) * (grads['db' + str(l+1)]**2)
            v_dW_corrected = v_dW / (1 - beta1)
            v_db_corrected = v_db / (1 - beta1)
            s_dW_corrected = s_dW / (1 - beta2)
            s_db_corrected = s_db / (1 - beta2)
            parameters['W' + str(l+1)] -= learning_rate * v_dW_corrected / (np.sqrt(s_dW_corrected) + epsilon)
            parameters['b' + str(l+1)] -= learning_rate * v_db_corrected / (np.sqrt(s_db_corrected) + epsilon)
    return parameters

def predict(X, parameters):
    AL, _ = forward_propagation(X, parameters)
    return np.argmax(AL, axis=0)

def accuracy(predictions, labels):
    return np.mean(predictions == np.argmax(labels, axis=0))


def to_categorical(labels, num_classes):
    # Initialize an array of zeros with shape (num_classes, num_samples)
    one_hot = np.zeros((num_classes, labels.shape[0]))
    # Set the corresponding index of each sample to 1
    one_hot[labels, np.arange(labels.shape[0])] = 1
    return one_hot.T  # Transpose the array to match the desired shape

# Preprocess the data
train_images = X_train / 255.0
test_images = X_test / 255.0
train_images = train_images.reshape(train_images.shape[0], -1).T
test_images = test_images.reshape(test_images.shape[0], -1).T

# Convert labels to one-hot encoding
train_labels = to_categorical(Y_train, num_classes=10).T
test_labels = to_categorical(Y_test, num_classes=10).T

# Define hyperparameters
layer_dims = [train_images.shape[0], 128, 10]
learning_rate = 0.01
num_epochs = 100

# Initialize parameters
parameters = initialize_parameters(layer_dims)

# Train the model with different optimization methods
optimization_methods = ["SGD", "Momentum", "Nesterov", "RMSprop", "Adam", "Nadam"]
for method in optimization_methods:
    print(f"\nTraining with {method} optimizer...")
    parameters = initialize_parameters(layer_dims)  # Re-initialize parameters for each optimization method
    for epoch in range(num_epochs):
        AL, caches = forward_propagation(train_images, parameters)
        cost = compute_cost(AL, train_labels)
        grads = backward_propagation(AL, train_labels, caches)
        parameters = update_parameters(parameters, grads, learning_rate, method=method)
        print(f"Epoch {epoch+1}/{num_epochs}, Cost: {cost:.4f}")

    # Evaluate the model
    train_predictions = predict(train_images, parameters)
    test_predictions = predict(test_images, parameters)
    train_acc = accuracy(train_predictions, train_labels)
    test_acc = accuracy(test_predictions, test_labels)
    print(f"Train Accuracy ({method}): {train_acc:.4f}")
    print(f"Test Accuracy ({method}): {test_acc:.4f}")



Training with SGD optimizer...
Epoch 1/100, Cost: 2.3021
Epoch 2/100, Cost: 2.3017
Epoch 3/100, Cost: 2.3013
Epoch 4/100, Cost: 2.3009
Epoch 5/100, Cost: 2.3006
Epoch 6/100, Cost: 2.3002
Epoch 7/100, Cost: 2.2998
Epoch 8/100, Cost: 2.2994
Epoch 9/100, Cost: 2.2990
Epoch 10/100, Cost: 2.2987
Epoch 11/100, Cost: 2.2983
Epoch 12/100, Cost: 2.2979
Epoch 13/100, Cost: 2.2975
Epoch 14/100, Cost: 2.2971
Epoch 15/100, Cost: 2.2967
Epoch 16/100, Cost: 2.2963
Epoch 17/100, Cost: 2.2959
Epoch 18/100, Cost: 2.2955
Epoch 19/100, Cost: 2.2951
Epoch 20/100, Cost: 2.2947
Epoch 21/100, Cost: 2.2943
Epoch 22/100, Cost: 2.2939
Epoch 23/100, Cost: 2.2935
Epoch 24/100, Cost: 2.2931
Epoch 25/100, Cost: 2.2927
Epoch 26/100, Cost: 2.2922
Epoch 27/100, Cost: 2.2918
Epoch 28/100, Cost: 2.2914
Epoch 29/100, Cost: 2.2909
Epoch 30/100, Cost: 2.2905
Epoch 31/100, Cost: 2.2901
Epoch 32/100, Cost: 2.2896
Epoch 33/100, Cost: 2.2892
Epoch 34/100, Cost: 2.2887
Epoch 35/100, Cost: 2.2882
Epoch 36/100, Cost: 2.2878
Epoch